In [1]:
#reading relevant packages
import pandas as pd
import folium
import geopandas as gpd
import os
import fiona
from shapely.geometry import shape
from shapely.geometry import Point
import matplotlib.pyplot as plt
import requests
import openpyxl
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
from scipy.stats import gaussian_kde